In [1]:
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
import pickle, os, glob
from EEG.info import info_exp
from EEG.converter import converter_mne
from EEG.preprocessing import aligner_exp
import mne
import numpy as np
import sys

In [2]:
root = "C:\\eeg\\01exp\\"
name = "20161129_DBS_001"

def load(path,name):
    with open(path + '/' + name, 'rb') as input:
        data = pickle.load(input)
        return data

In [3]:
test = aligner_exp(root + name)

In [4]:
raw = test.getdata(mode='raw')

In [5]:
raw_train = raw['data']['data_train']
raw_events = raw['data']['states_train']

In [6]:
import neo
import mne

mne_info = mne.create_info(ch_names = raw['info'].chanels_names, 
                               sfreq = raw['info'].frequency,
                               ch_types = raw['info'].ch_types, 
                               montage = raw['info'].montage)

raw_mne = mne.io.RawArray(raw_train, mne_info)

In [7]:
def create_events(states_full):
    xa = np.hstack([[0], states_full[0], [0]])
    xa1 = (xa != 0) + 0
    xadf = np.diff(xa1)
    starts = np.where(xadf == 1)[0]
    stops_p1 = np.where(xadf == -1)[0]
    lens = stops_p1 - starts
    codes = states_full[0, starts]
    trl = np.zeros((len(starts), 3))
    start = 0
    min_len = min(lens)
    for i in range(0, len(lens)):
        trl[i] = [start, 0, codes[i]]
        start = start + lens[i]
    return trl.astype(int)

events = create_events(raw_events)

In [160]:
def get_labels(labels_names):
    labels = dict.fromkeys(labels_names)
    for idx, i in enumerate(labels):
        labels[i] = idx
    labels.pop(u'удалено', None)
    return labels

labels = get_labels(raw['info'].labels_names)
print labels

epochs = mne.Epochs(raw_mne,events,event_id=labels,add_eeg_ref=False, preload = True, tmin = 0, tmax = 4.499, proj=False, decim=1)

{u'\u043b\u0435\u0432\u0430\u044f': 1, u'\u043e\u0442\u0434\u044b\u0445': 2, u'\u043f\u0440\u0430\u0432\u0430\u044f': 3}


In [161]:
epochs

<Epochs  |  n_events : 45 (all good), tmin : 0.0 (s), tmax : 4.499 (s), baseline : (None, 0), ~46.4 MB, data loaded,
 u'\u043b\u0435\u0432\u0430\u044f': 15, u'\u043e\u0442\u0434\u044b\u0445': 15, u'\u043f\u0440\u0430\u0432\u0430\u044f': 15>

In [162]:
data_epochs = epochs.get_data()
data_events = epochs.events

In [163]:
data_epochs.shape

(45L, 30L, 4500L)

In [164]:
data_epochs[0][0][200:205]

array([ -1.40737825e-06,  -1.76597306e-06,  -2.11547823e-06,
        -2.44756155e-06,  -2.75401991e-06])

In [165]:
test = load(root + name,'exp_data_aligned.cls')

In [166]:
test.getdata()['data']['data_train'][0][0][200]

4.2775411371365863e-06

In [167]:
raw_train[0]

array([ -1.16324102e-10,  -9.74026060e-10,  -4.02917729e-09, ...,
        -1.06943455e-06,  -4.77476051e-07,   9.94954702e-08])

In [168]:
np.where(raw_train == test.getdata()['data']['data_train'][0][0][0])

(array([0], dtype=int64), array([250], dtype=int64))

In [169]:
np.where(data_epochs == test.getdata()['data']['data_train'][0][0][0])

(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))

In [177]:
data_epochs[0][0]

array([  0.00000000e+00,  -8.57701958e-10,  -3.91285319e-09, ...,
         3.57431608e-06,   3.74122967e-06,   3.83215316e-06])

In [178]:
raw_train[0][0]

-1.1632410239334387e-10

In [179]:
np.where(data_epochs == raw_train[0][0])

(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))

In [181]:
data_epochs.shape

(45L, 30L, 4500L)

In [186]:
test.getdata()['data']['states_train']

array([ 2.,  2.,  1.,  3.,  2.,  2.,  2.,  3.,  3.,  3.,  1.,  3.,  1.,
        3.,  3.,  2.,  1.,  1.,  3.,  2.,  2.,  3.,  3.,  1.,  1.,  3.,
        3.,  1.,  2.,  1.,  2.,  2.,  1.,  3.,  3.,  2.,  2.,  1.,  1.,
        1.,  2.,  1.,  1.,  3.,  2.])

In [187]:
import scipy.signal as spsig
def data_prep(data, sfreq, fmin, fmax):
    [b_high, a_high] = spsig.butter(4, float(fmin) / (sfreq / 2), 'high')
    [b_low, a_low] = spsig.butter(4, float(fmax) / (sfreq / 2), 'low')

    chunk_high = spsig.lfilter(b_high, a_high, data)
    chunk_low = spsig.lfilter(b_low, a_low, chunk_high)
    return chunk_low

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from mne.decoding import CSP
from sklearn.multiclass import OneVsRestClassifier

Y_train = epochs.events[:,-1]
X_train = epochs.get_data()
X_train = data_prep(X_train, 1000, 6, 16)

# train classifier
svc = LinearDiscriminantAnalysis()
csp = CSP(n_components=4, reg='ledoit_wolf', log=True)


X_train = csp.fit_transform(X_train, Y_train)
svc.fit(X_train, Y_train)

LDA_scores = []
for i in range(0, len(test_list)):
    X_test = data_prep(test_list[i]['X_test'], 1000, 6, 16)
    X_test = csp.transform(X_test)
    LDA_scores.append(svc.score(X_test, test_list[i]['Y_test']))

print "MAX:", np.max(LDA_scores)
print "MEAN:", np.mean(LDA_scores)
print "MIN:", np.min(LDA_scores)
print "DISP:", np.std(LDA_scores)